<a href="https://colab.research.google.com/github/rjnakano/manejo-inventarios-pdg/blob/main/Notebooks/clusterboot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install --upgrade kmodes

Requirement already up-to-date: kmodes in /usr/local/lib/python3.7/dist-packages (0.11.0)


In [2]:
from google.colab import drive
import pathlib
import pandas as pd
import numpy as np
from pylab import *

from sklearn.preprocessing import StandardScaler

from kmodes.kprototypes import KPrototypes

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx

import seaborn as sns
%matplotlib inline
sns.set_context('poster')
sns.set_style('white')
sns.set_color_codes()

from plotnine import *
import plotnine

In [3]:
plot_kwds = {'alpha' : 0.5, 's' : 1, 'linewidths':0}

In [4]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
# Dataset de inventarios sin dimensiones de los productos
data1 = pd.read_csv("/content/gdrive/MyDrive/TDGdata/df042421.csv")

In [6]:
data1['AISLE']=data1['AISLE'].astype(str)

In [7]:
data1.dtypes

AISLE        object
IG           object
HTS          object
PLANNING     object
ABC          object
UOM          object
CAT          object
COSTX       float64
QTYVAR      float64
dtype: object

In [8]:
data1.head(5)

,AISLE,IG,HTS,PLANNING,ABC,UOM,CAT,COSTX,QTYVAR
0,17,IG56,HTS119,1/1,B,PC,CAT17,-0.029671,-0.037167
1,17,IG56,HTS119,1/1,B,PC,CAT17,0.092655,-0.006905
2,17,IG56,HTS119,1/1,C,PC,CAT17,0.266138,0.020477
3,17,IG56,HTS119,1/1,B,PC,CAT17,-0.024812,-0.044242
4,15,IG56,HTS119,1/1,C,PC,CAT17,-0.077628,-0.046455


In [9]:
# scikit-learn bootstrap
from sklearn.utils import resample

In [10]:
data1.shape[0]

16244

In [11]:
# prepare bootstrap sample
boot = resample(data1, replace=True, n_samples=data1.shape[0], random_state=1)

In [12]:
boot.head(5)

,AISLE,IG,HTS,PLANNING,ABC,UOM,CAT,COSTX,QTYVAR
13349,13,IG46,HTS119,1/1,C,PR,CAT17,0.217599,0.002996
235,17,IG45,HTS24,1/1,A,BG,CAT17,0.005390,-0.039992
12172,32,IG61,HTS6,1/1,A,BG,CAT14,0.354152,-0.033191
5192,2,IG38,HTS61,1/1,A,SQFT,CAT8,-0.052529,-0.143812
16127,22,IG17,HTS95,1/0,C,BX,CAT12,0.349906,-0.033191


In [13]:
boot.shape

(16244, 9)

In [14]:
catColumnsPos = [data1.columns.get_loc(col) for col in list(data1.select_dtypes('object').columns)]

In [15]:
catColumnsPos

[0, 1, 2, 3, 4, 5, 6]

In [16]:
# Fit the cluster 
k = 4
gamma = 0.5
import time
start_time = time.time()
kprototype = KPrototypes(n_jobs = -1, n_clusters = k, init = 'Huang', gamma = gamma, random_state = 0)
kprototype.fit_predict(data1, categorical = catColumnsPos)
print("--- %s seconds ---" % (time.time() - start_time))

--- 56.17043733596802 seconds ---


In [17]:
print(kprototype.cluster_centroids_)

[['0.002726388751717149' '-0.03595785577407407' '2' 'IG45' 'HTS119' '1/1'
  'A' 'BX' 'CAT17']
 ['-3.6487292748298596' '-0.11901373330295531' '37' 'IG64' 'HTS15' '1/1'
  'A' 'BX' 'CAT14']
 ['1.784357513065899' '36.7170829887051' '2' 'IG38' 'HTS61' '1/1' 'A'
  'SQFT' 'CAT8']
 ['2.79799290946075' '0.5053389823974501' '37' 'IG64' 'HTS15' '1/1' 'A'
  'BX' 'CAT14']]


In [18]:
data1['Cluster'] = kprototype.labels_

In [19]:
boot.dtypes

AISLE        object
IG           object
HTS          object
PLANNING     object
ABC          object
UOM          object
CAT          object
COSTX       float64
QTYVAR      float64
dtype: object

In [20]:
# Fit the cluster 
k = 4
gamma = 0.5
import time
start_time = time.time()
kprototypebs = KPrototypes(n_jobs = -1, n_clusters = k, init = 'Huang', gamma = gamma, random_state = 0)
kprototypebs.fit_predict(boot, categorical = catColumnsPos)
print("--- %s seconds ---" % (time.time() - start_time))

--- 58.65824341773987 seconds ---


In [21]:
print(kprototypebs.cluster_centroids_)

[['0.05565383927700991' '-0.022394882604889192' '12' 'IG46' 'HTS119'
  '1/1' 'A' 'PC' 'CAT17']
 ['-0.06438038500544505' '-0.053530134537021434' '2' 'IG14' 'HTS15' '1/1'
  'C' 'BX' 'CAT8']
 ['-4.199122494971278' '-0.11922423118019004' '37' 'IG61' 'HTS15' '1/1'
  'A' 'BX' 'CAT14']
 ['2.9289436099474213' '1.0022811424258908' '37' 'IG64' 'HTS15' '1/1' 'A'
  'BX' 'CAT14']]


In [22]:
boot['Cluster'] = kprototypebs.labels_

In [23]:
from sklearn.metrics import jaccard_score

In [27]:
jaccard_score(data1['Cluster'], boot['Cluster'], average=None)

array([0.4904868 , 0.02772796, 0.        , 0.02278703])

In [37]:
for i in range(4):
  print('Cluster (data1)',i,'\n');
  print(data1[data1['Cluster']==i].describe(),'\n');
  print('Cluster (boot)',i,'\n');
  print(boot[boot['Cluster']==i].describe(),'\n');

Cluster (data1) 0 

              COSTX        QTYVAR  Cluster
count  15143.000000  15143.000000  15143.0
mean       0.002726     -0.035958      0.0
std        0.395524      0.197223      0.0
min       -1.932898     -0.143812      0.0
25%       -0.109056     -0.089049      0.0
50%        0.023141     -0.035831      0.0
75%        0.133249     -0.030053      0.0
max        1.583515      6.354699      0.0 

Cluster (boot) 0 

             COSTX       QTYVAR  Cluster
count  8280.000000  8280.000000   8280.0
mean      0.055654    -0.022395      0.0
std       0.404553     0.215577      0.0
min      -2.050928    -0.143812      0.0
25%      -0.034092    -0.056191      0.0
50%       0.040643    -0.033191      0.0
75%       0.153810    -0.028744      0.0
max       1.855586     4.182446      0.0 

Cluster (data1) 1 

            COSTX      QTYVAR  Cluster
count  483.000000  483.000000    483.0
mean    -3.648729   -0.119014      1.0
std      1.649935    0.031328      0.0
min     -8.415911   -0.14

In [ ]:
#sklearn.metrics.jaccard_score(y_true, y_pred, *, labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')[source]